In [ ]:
import sys, os
# Ensure project root (containing 'tps' package) is on sys.path when running the notebook
candidates = [os.getcwd(), os.path.abspath(os.path.join(os.getcwd(), '..')), os.path.abspath(os.path.join(os.getcwd(), '..', '..'))]
for p in candidates:
    if os.path.exists(os.path.join(p, 'tps')):
        if p not in sys.path:
            sys.path.insert(0, p)
            print('Inserted into sys.path:', p)
            break


# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [11]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [12]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://max.de.wilde.web.ulb.be/camille/"
response = requests.get(root_url, headers=headers,verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a"):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

c:\Users\sophi\tac\tac_venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'max.de.wilde.web.ulb.be'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
# Affichage du nombre d'articles récupérés
len(articles)

51

In [14]:
# Affichage des 10 premières entrées
articles[:10]

[['https://max.de.wilde.web.ulb.be/camille/KB_JB230_1892-08-07_01-0003.pdf',
  'KB_JB230_1892-08-07_01-0003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB427_1920-01-10_01-00004.pdf',
  'KB_JB427_1920-01-10_01-00004.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB555_1836-02-08_01-00002.pdf',
  'KB_JB555_1836-02-08_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB638_1860-05-21_01-00002.pdf',
  'KB_JB638_1860-05-21_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB773_1918-11-30_01-00002.pdf',
  'KB_JB773_1918-11-30_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB838_1887-12-28_01-00003.pdf',
  'KB_JB838_1887-12-28_01-00003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB230_1903-10-16_01-0002.pdf',
  'KB_JB230_1903-10-16_01-0002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB427_1933-01-04_01-00003.pdf',
  'KB_JB427_1933-01-04_01-00003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB555_1899-01-19_01-00

## Création d'un dataframe avec les liens et les titres des articles


In [15]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://max.de.wilde.web.ulb.be/camille/KB_JB2...,KB_JB230_1892-08-07_01-0003.pdf
1,https://max.de.wilde.web.ulb.be/camille/KB_JB4...,KB_JB427_1920-01-10_01-00004.pdf
2,https://max.de.wilde.web.ulb.be/camille/KB_JB5...,KB_JB555_1836-02-08_01-00002.pdf
3,https://max.de.wilde.web.ulb.be/camille/KB_JB6...,KB_JB638_1860-05-21_01-00002.pdf
4,https://max.de.wilde.web.ulb.be/camille/KB_JB7...,KB_JB773_1918-11-30_01-00002.pdf
5,https://max.de.wilde.web.ulb.be/camille/KB_JB8...,KB_JB838_1887-12-28_01-00003.pdf
6,https://max.de.wilde.web.ulb.be/camille/KB_JB2...,KB_JB230_1903-10-16_01-0002.pdf
7,https://max.de.wilde.web.ulb.be/camille/KB_JB4...,KB_JB427_1933-01-04_01-00003.pdf
8,https://max.de.wilde.web.ulb.be/camille/KB_JB5...,KB_JB555_1899-01-19_01-00003.pdf
9,https://max.de.wilde.web.ulb.be/camille/KB_JB6...,KB_JB638_1902-12-20_01-00002.pdf


In [16]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [17]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

ConnectTimeout: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille/KB_JB230_1892-08-07_01-0003.pdf (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B91CE21F90>, 'Connection to max.de.wilde.web.ulb.be timed out. (connect timeout=None)'))

### Nettoyage du texte à l'aide d'expressions régulières

In [18]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


NameError: name 'content' is not defined

### Création d'un fichier avec le contenu de l'article


In [19]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

NameError: name 'clean_content' is not defined

## Téléchargement automatique des fichiers PDF

Le bloc suivant recherche tous les liens se terminant par .pdf sur la page racine et télécharge chaque fichier dans le dossier `../data/PDF`. Les fichiers déjà présents sont ignorés.


In [ ]:
,
,
,
,
,
,
,
,
,
,
500
502
503
504
,

c:\Users\sophi\tac\tac_venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'max.de.wilde.web.ulb.be'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 51 PDF links
Failed to download https://max.de.wilde.web.ulb.be/camille/KB_JB230_1892-08-07_01-0003.pdf: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille/KB_JB230_1892-08-07_01-0003.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Failed to download https://max.de.wilde.web.ulb.be/camille/KB_JB230_1903-10-16_01-0002.pdf: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille/KB_JB230_1903-10-16_01-0002.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Failed to download https://max.de.wilde.web.ulb.be/camille/KB_JB230_1913-07-05_01-0001.pdf: HTTPSConnectionPool(host='max.de.wilde.web.ulb.be', port=443): Max retries exceeded with url: /camille/